# New Service subscription for Accounts

In 3Scale at version 2.14 you cannot directly subscribe all Accounts to a new Service you created. Instead of going manually through all accounts to to enable this subscription, we will use the API with a specific recipe to programmatically subscribe an Account to a Service:

- Retrieve the Account id to which we need to enable the subscription.
- Retrieve the Service and the associated Service Plan we want the subscription for.
- Create a dummy application based on this Service Plan in the target Account. This will automatically subscribe the account to the Service.
- Delete the dummy application immediately afterwards. The subscription to the Service will remain active. Voilà!

To enable a Service for all accounts, you can simply loop the recipe over all accounts!

### Init

In [1]:
import os
import requests
import xmltodict
from dotenv import load_dotenv

In [2]:
load_dotenv()
backend_address = os.environ.get('BACKEND_ADDRESS')
access_token = os.environ.get('ACCESS_TOKEN')

### Get the Accounts

Note: those are organization accounts, not user accounts within an organization

In [3]:
params = {
    "access_token": access_token
}
headers = {
    "accept": "*/*"
}

api_url = f"{backend_address}/admin/api/accounts.xml"

response = requests.get(api_url, params=params, headers=headers)

accounts = xmltodict.parse(response.content)

print(f"{len(accounts['accounts']['account'])} accounts found")

342 accounts found


In [4]:
# Display a specific account
print(accounts["accounts"]["account"][1])

{'id': '7', 'created_at': '2024-08-14T09:07:14-04:00', 'updated_at': '2024-08-14T09:07:14-04:00', 'state': 'approved', 'org_name': 'gmoutier@redhat.com', 'extra_fields': None, 'monthly_billing_enabled': 'true', 'monthly_charging_enabled': 'true', 'credit_card_stored': 'false', 'plans': {'plan': [{'@default': 'true', 'id': '6', 'name': 'Default', 'type': 'account_plan', 'state': 'hidden', 'approval_required': 'false', 'setup_fee': '0.0', 'cost_per_month': '0.0', 'trial_period_days': None, 'cancellation_period': '0'}, {'@default': 'true', 'id': '9', 'name': 'Default', 'type': 'service_plan', 'state': 'published', 'approval_required': 'false', 'setup_fee': '0.0', 'cost_per_month': '0.0', 'trial_period_days': None, 'cancellation_period': '0', 'service_id': '3'}, {'@default': 'true', 'id': '10', 'name': 'Default', 'type': 'service_plan', 'state': 'published', 'approval_required': 'false', 'setup_fee': '0.0', 'cost_per_month': '0.0', 'trial_period_days': None, 'cancellation_period': '0', 'se

In [5]:
# Set the account to use in the next requests
account_id = 7 # org gmoutier@redhat.com

### Get the Services

In [3]:
params = {
    "access_token": access_token
}
headers = {
    "accept": "*/*"
}

api_url = f"{backend_address}/admin/api/services.xml"

response = requests.get(api_url, params=params, headers=headers)

services = xmltodict.parse(response.content)

print(f"{len(services['services']['service'])} services found")

15 services found


In [4]:
# Display available Services    
for service in services["services"]["service"]:
    print(f"Service id: {service['id']}")
    print(f"Service name: {service['name']}")
    print("---")

Service id: 3
Service name: Granite-8B-Code-Instruct
---
Service id: 4
Service name: Mistral-7B-Instruct-v0.3
---
Service id: 5
Service name: Nomic-embed-text-v1.5
---
Service id: 6
Service name: Granite-3.1-8B-Instruct
---
Service id: 7
Service name: Docling
---
Service id: 8
Service name: Granite-8B-Lab-v1
---
Service id: 9
Service name: Granite-Embedding-278m-multilingual
---
Service id: 10
Service name: DeepSeek-R1-Distill-Qwen-14B
---
Service id: 11
Service name: StableDiffusion-XL
---
Service id: 12
Service name: DeepSeek-R1-Distill-Qwen-14B-W4A16
---
Service id: 13
Service name: Granite-3.1-8B-Instruct-W4A16
---
Service id: 14
Service name: Llama-3.1-8B-Instruct
---
Service id: 15
Service name: Granite Guardian 3.1 2B
---
Service id: 16
Service name: Stable Diffusion Safety Checker
---
Service id: 17
Service name: Mixtral-8x7B-Instruct-v0.1
---


In [5]:
# Set the Service to use in the next requests
service_id = 8 # Granite-8B-Lab-v1

### Get the Service Plans

We need the Service Plan id to use when creating a dummy application.

In [6]:
params = {
    "access_token": access_token
}
headers = {
    "accept": "*/*"
}

api_url = f"{backend_address}/admin/api/application_plans.xml"

response = requests.get(api_url, params=params, headers=headers)

plans = xmltodict.parse(response.content)

print(f"{len(plans['plans']['plan'])} application plans found")

15 application plans found


In [7]:
# Display available Plans
for plan in plans["plans"]["plan"]:
    print(f"Plan id: {plan['id']}")
    print(f"Plan name: {plan['name']}")
    print(f"Service id: {plan['service_id']}")
    print("---")

Plan id: 22
Plan name: Standard Plan
Service id: 9
---
Plan id: 38
Plan name: Standard
Service id: 17
---
Plan id: 36
Plan name: Standard
Service id: 16
---
Plan id: 34
Plan name: Standard
Service id: 15
---
Plan id: 32
Plan name: Standard
Service id: 14
---
Plan id: 30
Plan name: Standard
Service id: 13
---
Plan id: 28
Plan name: Standard
Service id: 12
---
Plan id: 26
Plan name: Standard Plan
Service id: 11
---
Plan id: 24
Plan name: Standard Plan
Service id: 10
---
Plan id: 20
Plan name: Standard Plan
Service id: 8
---
Plan id: 18
Plan name: Standard Plan
Service id: 7
---
Plan id: 16
Plan name: Standard Plan
Service id: 6
---
Plan id: 14
Plan name: Standard Plan
Service id: 3
---
Plan id: 13
Plan name: Standard Plan
Service id: 4
---
Plan id: 12
Plan name: Standard Plan
Service id: 5
---


In [11]:
# Set the Plan to use in the next requests
plan_id = 20

### Create a dummy Application in the selected Account, using the selected Plan

In [19]:
payload = {
    "access_token": access_token,
    "plan_id": "20",
    "name": "dummy2",
}
headers = {
    "accept": "*/*",
    "Content-Type": "application/x-www-form-urlencoded",
}

api_url = f"{backend_address}/admin/api/accounts/{account_id}/applications.xml"

response = requests.post(api_url, data=payload, headers=headers)

application = xmltodict.parse(response.content)

# Retrieve the application id (to delete it afterwards)
application_id = application["application"]["id"]
print(f"New application created, id: {application_id}")

New application created, id: 2565


### Delete dummy app

In [20]:
params = {
    "access_token": access_token,
}
headers = {
    "accept": "*/*",
}

api_url = f"{backend_address}/admin/api/accounts/{account_id}/applications/{application_id}.xml"

response = requests.delete(api_url, headers=headers, params=params)

if response.status_code == 200:
    print(f"Application {application_id} deleted")
else:
    print(f"Error deleting application {application_id}")
    print(response.content)

Application 2565 deleted
